### 1. Install and import the required packages

In [16]:
!pip install transformers sentence-transformers datasets

16329.03s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [4]:
!pip install protobuf==3.20.*

In [44]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, models
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
import datetime
import random
import numpy as np
import pandas as pd

### 2. Use Google Colab's GPU for training

In [45]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA L4


### **3.** Load and preview the Semantic Textual Similarity Benchmark (STSB) dataset

In [3]:
# from datasets import load_dataset
# dataset = load_dataset("csv", data_files="Match Manually NM.csv")

In [4]:
# Load the English version of the STSB dataset
# dataset = load_dataset("stsb_multi_mt", "en")

In [5]:
# print(dataset)

In [6]:
# print("A sample from the STSB dataset's training split:")
# print(dataset['train'][98])

### **4.** Define the dataset loader class


In [46]:
# Instantiate the BERT tokenizer
# You can use larger variants of the model, here we're using the base model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

## loading the training data and testing dataset over here

In [47]:
import pandas as pd
test_df=pd.read_csv("NM_Concat_balanced.csv")

In [48]:
test_df.drop_duplicates(inplace=True)

In [49]:
test_df.dropna(subset=["Match Flag (1/0)","name1","name2",], inplace=True)

In [50]:
# test_df.dropna(subset=["Match Flag (1/0)"], inplace=True)

In [51]:
test_df.isnull().sum()

id                  0
name1               0
name2               0
Match Flag (1/0)    0
dtype: int64

In [52]:
test_df.shape

(10888, 4)

In [53]:
# test_df=test_df.head(10000)
test_df

,id,name1,name2,Match Flag (1/0)
0,49527067,BEBI,BABY,1
1,48117328,TEJA,THEJU,1
2,45469064,UJMA,UZMA,1
3,50245724,YASH,Yesh,1
4,50660565,AAFAK,AFAQ S/O HANEEF,1
...,...,...,...,...
10884,49067292,ADITYA KUMAR INDRI,ADITYA TRADERS,0
10885,47211780,ALOK KUMAR JAISWAL,ALOK SARI CENTER,0
10886,49870635,AMIT KUMAR AGRAWAL,AMIT AGENCY,0
10887,47196423,ANANDA MOHAN GHOSH,GHOSH BASTRALAYA,0


In [54]:
test_df['Match Flag (1/0)'].value_counts()

Match Flag (1/0)
1    5889
0    4999
Name: count, dtype: int64

In [55]:
test_df.to_csv("Match Manually NM_Balanced.csv",index=False)

In [64]:
test_df

,id,name1,name2,Match Flag (1/0)
0,49527067,BEBI,BABY,1
1,48117328,TEJA,THEJU,1
2,45469064,UJMA,UZMA,1
3,50245724,YASH,Yesh,1
4,50660565,AAFAK,AFAQ S/O HANEEF,1
...,...,...,...,...
10884,49067292,ADITYA KUMAR INDRI,ADITYA TRADERS,0
10885,47211780,ALOK KUMAR JAISWAL,ALOK SARI CENTER,0
10886,49870635,AMIT KUMAR AGRAWAL,AMIT AGENCY,0
10887,47196423,ANANDA MOHAN GHOSH,GHOSH BASTRALAYA,0


In [76]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="Match Manually NM_Balanced.csv")

In [77]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'name1', 'name2', 'Match Flag (1/0)'],
        num_rows: 10888
    })
})

In [67]:
# dataset = dataset['train']

In [75]:
# dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'name1', 'name2', 'Match Flag (1/0)'],
        num_rows: 1960
    })
    test: Dataset({
        features: ['id', 'name1', 'name2', 'Match Flag (1/0)'],
        num_rows: 218
    })
})

In [68]:
# dataset = dataset.train_test_split(test_size=0.2)

In [69]:
# dataset = dataset['test'].train_test_split(test_size=0.1)

In [70]:
from datasets import DatasetDict

In [78]:
# First, split the dataset into 80% train and 20% test+valid
train_testvalid = dataset['train'].train_test_split(test_size=0.2)

# Now split the 20% into 10% test and 10% validation
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)  # 50% of the 20% becomes 10% of the total

# Create the DatasetDict with train, test, and valid sets
ds = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']
})


In [79]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'name1', 'name2', 'Match Flag (1/0)'],
        num_rows: 8710
    })
    test: Dataset({
        features: ['id', 'name1', 'name2', 'Match Flag (1/0)'],
        num_rows: 1089
    })
    valid: Dataset({
        features: ['id', 'name1', 'name2', 'Match Flag (1/0)'],
        num_rows: 1089
    })
})

In [25]:
# dataset=ds

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'merchant_type', 'created_at', 'status', 'docType', 'name', 'pan_createdAt', 'pan_status', 'beneficiary_name', 'bank_status', 'account_number', 'szFlag', 'hvFlag', 'name1', 'name2', 'dsScore', 'flFlag', 'dsFlag', 'szScore', 'flScore', 'flag', 'SCORE', 'Match Flag (1/0)'],
        num_rows: 5528
    })
    test: Dataset({
        features: ['id', 'merchant_type', 'created_at', 'status', 'docType', 'name', 'pan_createdAt', 'pan_status', 'beneficiary_name', 'bank_status', 'account_number', 'szFlag', 'hvFlag', 'name1', 'name2', 'dsScore', 'flFlag', 'dsFlag', 'szScore', 'flScore', 'flag', 'SCORE', 'Match Flag (1/0)'],
        num_rows: 615
    })
})

In [86]:
class STSBDataset(torch.utils.data.Dataset):

    def __init__(self, dataset):

        # Normalize the similarity scores in the dataset
        similarity_scores = [i['Match Flag (1/0)'] for i in dataset]
        self.normalized_similarity_scores = similarity_scores
        self.first_sentences = [i['name1'] for i in dataset]
        self.second_sentences = [i['name2'] for i in dataset]
        self.concatenated_sentences = [[str(x), str(y)] for x,y in zip(self.first_sentences, self.second_sentences)]

    def __len__(self):
        return len(self.concatenated_sentences)

    def get_batch_labels(self, idx):
        return torch.tensor(self.normalized_similarity_scores[idx])

    def get_batch_texts(self, idx):
        return tokenizer(self.concatenated_sentences[idx], padding='max_length', max_length=128, truncation=True, return_tensors="pt")

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y


def collate_fn(texts):
    input_ids = texts['input_ids']
    attention_masks = texts['attention_mask']
    features = [{'input_ids': input_id, 'attention_mask': attention_mask}
                for input_id, attention_mask in zip(input_ids, attention_masks)]
    return features

### 5. Define the model class based on BERT

In [87]:
class BertForSTS(torch.nn.Module):

    def __init__(self):
        super(BertForSTS, self).__init__()
        self.bert = models.Transformer('bert-base-uncased', max_seq_length=128)
        self.pooling_layer = models.Pooling(self.bert.get_word_embedding_dimension())
        self.sts_bert = SentenceTransformer(modules=[self.bert, self.pooling_layer])

    def forward(self, input_data):
        output = self.sts_bert(input_data)['sentence_embedding']
        return output

In [88]:
# Instantiate the model and move it to GPU
model = BertForSTS()
model.to(device)

BertForSTS(
  (bert): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (pooling_layer): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (sts_bert): SentenceTransformer(
    (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  )
)

### 6. Define the Cosine Similarity loss function

In [89]:
class CosineSimilarityLoss(torch.nn.Module):

    def __init__(self,  loss_fn=torch.nn.MSELoss(), transform_fn=torch.nn.Identity()):
        super(CosineSimilarityLoss, self).__init__()
        self.loss_fn = loss_fn
        self.transform_fn = transform_fn
        self.cos_similarity = torch.nn.CosineSimilarity(dim=1)

    def forward(self, inputs, labels):
        emb_1 = torch.stack([inp[0] for inp in inputs])
        emb_2 = torch.stack([inp[1] for inp in inputs])
        outputs = self.transform_fn(self.cos_similarity(emb_1, emb_2))
        return self.loss_fn(outputs, labels.squeeze())

### 7. Prepare the training and validation data split

In [90]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'name1', 'name2', 'Match Flag (1/0)'],
        num_rows: 10888
    })
})

In [91]:
dataset['train']

Dataset({
    features: ['id', 'name1', 'name2', 'Match Flag (1/0)'],
    num_rows: 10888
})

In [92]:
val_ds

In [93]:
train_ds = STSBDataset(dataset['train'])
val_ds = STSBDataset(dataset['test'])

# train_ds=load_dataset("csv", data_files="Train_data.csv")
# val_ds=load_dataset("csv", data_files="Test_data.csv")


# Create a 90-10 train-validation split.
train_size = len(train_ds)
val_size = len(val_ds)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

# # Assign the train, validation, and test datasets based on the updated splits
# train_ds = STSBDataset(dataset['train'])
# val_ds = STSBDataset(dataset['valid'])
# test_ds = STSBDataset(dataset['test'])

# # Uncomment the following lines if you need to load datasets from CSV files
# # train_ds = load_dataset("csv", data_files="Train_data.csv")
# # val_ds = load_dataset("csv", data_files="Validation_data.csv")
# # test_ds = load_dataset("csv", data_files="Test_data.csv")

# # Get the size of training, validation, and test datasets
# train_size = len(train_ds)
# val_size = len(val_ds)
# test_size = len(test_ds)

# # Print the number of samples in training, validation, and test sets
# print('{:>5,} training samples'.format(train_size))
# print('{:>5,} validation samples'.format(val_size))
# print('{:>5,} testing samples'.format(test_size))

KeyError: 'test'

In [38]:
batch_size = 8

train_dataloader = DataLoader(
            train_ds,  # The training samples.
            num_workers = 4,
            batch_size = batch_size, # Use this batch size.
            shuffle=True # Select samples randomly for each batch
        )

validation_dataloader = DataLoader(
            val_ds,
            num_workers = 4,
            batch_size = batch_size # Use the same batch size
        )


# batch_size = 8

# # Create the DataLoader for the training dataset
# train_dataloader = DataLoader(
#     train_ds,  # The training samples.
#     num_workers=4,
#     batch_size=batch_size,  # Use this batch size.
#     shuffle=True  # Select samples randomly for each batch
# )

# # Create the DataLoader for the validation dataset
# validation_dataloader = DataLoader(
#     val_ds,  # The validation samples.
#     num_workers=4,
#     batch_size=batch_size  # Use the same batch size
# )

# # Create the DataLoader for the test dataset
# test_dataloader = DataLoader(
#     test_ds,  # The test samples.
#     num_workers=4,
#     batch_size=batch_size  # Use the same batch size
# )

### 8. Define the Optimizer and Scheduler

In [39]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-6)

In [40]:
epochs = 1

# Total number of training steps is [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

### 9. Define a helper function for formatting the elapsed training time as `hh:mm:ss`

In [41]:
# Takes a time in seconds and returns a string hh:mm:ss
def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

### 10. Define the training function, and start the training loop

In [42]:
def train():
  seed_val = 42

  criterion = CosineSimilarityLoss()
  criterion = criterion.to(device)

  random.seed(seed_val)
  torch.manual_seed(seed_val)

  # We'll store a number of quantities such as training and validation loss,
  # validation accuracy, and timings.
  training_stats = []
  total_t0 = time.time()

  for epoch_i in range(0, epochs):

      # ========================================
      #               Training
      # ========================================

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      t0 = time.time()

      total_train_loss = 0

      model.train()

      # For each batch of training data...
      for train_data, train_label in tqdm(train_dataloader):

          train_data['input_ids'] = train_data['input_ids'].to(device)
          train_data['attention_mask'] = train_data['attention_mask'].to(device)

          train_data = collate_fn(train_data)
          model.zero_grad()

          output = [model(feature) for feature in train_data]

          loss = criterion(output, train_label.to(device))
          total_train_loss += loss.item()

          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          optimizer.step()
          scheduler.step()


      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)

      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("")
      print("  Average training loss: {0:.5f}".format(avg_train_loss))
      print("  Training epoch took: {:}".format(training_time))

      # ========================================
      #               Validation
      # ========================================

      print("")
      print("Running Validation...")

      t0 = time.time()

      model.eval()

      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0

      # Evaluate data for one epoch
      for val_data, val_label in tqdm(validation_dataloader):

          val_data['input_ids'] = val_data['input_ids'].to(device)
          val_data['attention_mask'] = val_data['attention_mask'].to(device)

          val_data = collate_fn(val_data)

          with torch.no_grad():
              output = [model(feature) for feature in val_data]

          loss = criterion(output, val_label.to(device))
          total_eval_loss += loss.item()

      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader)

      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)

      print("  Validation Loss: {0:.5f}".format(avg_val_loss))
      print("  Validation took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )

  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

  return model, training_stats

In [43]:
model, training_stats = train()


======== Epoch 1 / 1 ========
Training...


  0%|          | 0/691 [00:00<?, ?it/s]


RuntimeError: Found dtype Long but expected Float

In [ ]:
# Create a DataFrame from our training statistics
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index
df_stats = df_stats.set_index('epoch')

# Display the table
df_stats

In [ ]:
test_dataset = load_dataset("csv", data_files="Test_data.csv")

In [ ]:
test_dataset

In [ ]:
# test_dataset = load_dataset("stsb_multi_mt", name="en", split="test")

# Prepare the data
first_sent = [i[2] for i in test_dataset]
second_sent = [i[3] for i in test_dataset]
full_text = [[str(x), str(y)] for x,y in zip(first_sent, second_sent)]

In [ ]:
model.eval()

In [ ]:
def predict_similarity(sentence_pair):
  test_input = tokenizer(sentence_pair, padding='max_length', max_length = 512, truncation=True, return_tensors="pt").to(device)
  test_input['input_ids'] = test_input['input_ids']
  test_input['attention_mask'] = test_input['attention_mask']
  del test_input['token_type_ids']
  output = model(test_input)
  sim = torch.nn.functional.cosine_similarity(output[0], output[1], dim=0).item()

  return sim

In [ ]:
test_dataset

In [ ]:
import pandas as pd

# Access the 'train' part of the dataset
test_dataset = dataset['train']

# Set model to evaluation mode
model.eval()

# List to store similarity scores along with sentence pairs, dsScore, and flScore
results = []

# Iterate over the test dataset
with torch.no_grad():  # Disable gradients for evaluation
    for i in range(len(test_dataset)):
        # Extract sentence pair from the dataset
        sentence_pair = [test_dataset[i]['name1'], test_dataset[i]['name2']]
        
        # Predict the similarity score for the sentence pair
        sim_score = predict_similarity(sentence_pair)
        
        # Extract dsScore and flScore from the dataset
        dscore = test_dataset[i]['dsScore']
        f1score = test_dataset[i]['flScore']
        
        # Append the sentence pair, similarity score, dsScore, and flScore to the results list
        results.append({
            'name1': sentence_pair[0],
            'name2': sentence_pair[1],
            'similarity_score': sim_score,
            'dsScore': dscore,
            'flScore': f1score
        })

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('finetuned_embedding_model_results.csv', index=False)

print("Results saved to 'finetuned_embedding_model_results.csv'")

In [ ]:
# Define your sentence pair for testing
sentence_pair = ["Sara", "Sara"]

# Call the predict_similarity function inside a no_grad context
with torch.no_grad():
    similarity_score = predict_similarity(sentence_pair)

print(f"Similarity score: {similarity_score}")

### Last but not least, save your model!

In [145]:
PATH = 'bert-sts.pt'
torch.save(model.state_dict(), PATH)

In [ ]:
# In order to load the model
# First, you have to create an instance of the model's class
# And use the saving path for the loading
# Don't forget to set the model to the evaluation state using .eval()
model = BertForSTS()
model.load_state_dict(torch.load(PATH))
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForSTS(
  (bert): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (pooling_layer): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (sts_bert): SentenceTransformer(
    (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  )
)